In [1]:
import glob
import os
import pickle
import numpy

import time

import pandas as pd

In [2]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
DATA_PATH="data/"

### Read processed RAG results

In [5]:
with open(DATA_PATH + "passage_paths_teste_normas_new_full_20250122_114126.pkl", "rb") as input_file:
    new_segmentation_processed = pickle.load(input_file)

### Read the complete list of references

In [18]:
with open(DATA_PATH + "llama_deduplicated_embedded_and_annotated_reference_titles_20250214_fixed_after_match.pkl", "rb") as input_file:
    final_deduplicated_references_list = pickle.load(input_file)

In [19]:
final_deduplicated_references_list

array(['AD PGFN nº 3 de 18 de setembro de 2008', 'ADI RFB nº 12, de 2016',
       'Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha',
       'Acórdão do RE nº 855.091/RS (Tema 808)',
       'Anexo ao Decreto nº 361, de 10 de dezembro de 1991',
       'Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008',
       'Ato Declaratório Cosar nº 47, de 27 de novembro de 2000',
       'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019',
       'Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024',
       'Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005',
       'Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016',
       'Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007',
       'Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016',
       'Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005',
       'Ato Declarat

#### Remove first entry which is a duplicate of the 6th one.

In [20]:
final_deduplicated_references_list = final_deduplicated_references_list[1:]

In [21]:
final_deduplicated_references_list.shape

(480,)

In [23]:
with open(DATA_PATH + "llama_deduplicated_embedded_and_annotated_reference_titles_20250217_remove_last_dup.pkl", "wb") as output_file:
    pickle.dump(final_deduplicated_references_list, output_file, pickle.HIGHEST_PROTOCOL)

### Read the extracted references filename to final references list convertion

In [6]:
with open(DATA_PATH + "extracted_reference_match_results_20250214.pkl", "rb") as input_file:
    references = pickle.load(input_file)

In [7]:
references.keys()

dict_keys(['exact_matches', 'multiple_matches', 'file_to_reference', 'already_extracted', 'still_to_extract'])

In [9]:
references['file_to_reference']

{'Ato Declaratório Cosar nº 47.txt': 'Ato Declaratório Cosar nº 47, de 27 de novembro de 2000',
 'Ato Declaratório Interpretativo RFB nº 1.txt': 'Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016',
 'Ato Declaratório Interpretativo SRF nº 2.txt': 'Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007',
 'Ato Declaratório Interpretativo SRF nº 24.txt': 'Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004',
 'Ato Declaratório Interpretativo SRF nº 5.txt': 'Ato Declaratório Interpretativo SRF nº 5, de 27 de abril de 2005',
 'Ato Declaratório PGFN nº 13.txt': 'Ato Declaratório PGFN Nº 13, de 20 de dezembro de 2011',
 'Ato Declaratório PGFN nº 14.txt': 'Ato Declaratório PGFN Nº 14, de 1º de dezembro de 2008',
 'Ato Declaratório PGFN nº 2.txt': 'Ato Declaratório PGFN Nº 2, de 10 de março de 2016',
 'Ato Declaratório PGFN nº 3.txt': 'Ato Declaratório PGFN nº 3, de 30 de março de 2016',
 'Ato Declaratório PGFN nº 6 DE 2006.txt': 'Ato Declaratório PGFN

### Read final list of Q&A document data

In [5]:
with open(DATA_PATH + "extracted_questions_with_embedded_references_20250212_final.pkl", "rb") as input_file:
    questions = pickle.load(input_file)

In [6]:
len(questions)

715

In [12]:
questions[504]

{'question_number': '505',
 'question_summary': 'VENDA DE REBANHO BOVINO - GANHO DE CAPITAL',
 'question_text': 'A receita auferida na venda de rebanho bovino anteriormente comprado, com permanência dias nos demais casos, é tributada na atividade rural?',
 'answer': ['Não. O tratamento tributário é o seguinte:',
  '1 - os rendimentos auferidos, na venda, são tributados como ganho de capital, se essa atividade não for',
  'exercida com habitualidade (Lei nº 7.713, de 1988, art. 3º, § 2º); ou',
  '2 - se houver habitualidade e fim especulativo de lucro, a pessoa física é considerada empresário (empresa',
  'individual) equiparado a pessoa jurídica, sendo seus lucros tributados nessa condição (Regulamento do',
  'Imposto sobre a Renda – RIR/2018, arts. 162, 623 e 624, aprovado pelo Decreto nº 9.580, de 22 de novembro',
  'de 2018).',
  '(Instrução Normativa SRF nº 83, de 11 de outubro de 2001, art. 4º, inciso II; e Solução de Consulta',
  'Cosit nº 147, de 24 de setembro de 2018)'],
 'ans

In [15]:
questions[0].keys()

dict_keys(['question_number', 'question_summary', 'question_text', 'answer', 'answer_cleaned', 'references', 'linked_questions', 'formatted_references', 'embedded_references', 'formatted_embedded_references'])

In [16]:
questions[0]['formatted_references']

[{'título': 'Lei nº 9.250, de 26 de dezembro de 1995',
  'artigos': [{'artigo': '25', 'incisos': [], 'parágrafos': []}]},
 {'título': 'Lei nº 9.779, de 19 de janeiro de 1999',
  'artigos': [{'artigo': '16', 'incisos': [], 'parágrafos': []}]},
 {'título': 'Lei nº 11.482, de 31 de maio de 2007',
  'artigos': [{'artigo': '1º', 'incisos': ['X'], 'parágrafos': ['único']}]},
 {'título': 'Lei nº 14.754, de 12 de dezembro de 2023',
  'artigos': [{'artigo': '8º', 'incisos': [], 'parágrafos': []},
   {'artigo': '11', 'incisos': [], 'parágrafos': []},
   {'artigo': '14', 'incisos': [], 'parágrafos': []}]},
 {'título': 'Instrução Normativa RFB nº 2.178, de 5 de março de 2024',
  'artigos': [{'artigo': '2º', 'incisos': [], 'parágrafos': []}]},
 {'título': 'Instrução Normativa RFB nº 2.180, de 11 de março de 2024',
  'artigos': [],
  'anexos': []}]

In [17]:
questions[0]['formatted_embedded_references']

[{'título': 'Lei nº 11.196, de 21 de novembro de 2005',
  'artigos': [{'artigo': '39', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Lei nº 14.754, de 12 de dezembro de 2023',
  'artigos': [{'artigo': '8', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Lei nº 14.754, de 2023',
  'artigos': [{'artigo': '10 a 13', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Lei nº 14.754, de 2023',
  'artigos': [{'artigo': '14', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 2.180, de 11 de março de 2024',
  'artigos': [{'artigo': '36', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 2.180, de 2024',
  'artigos': [{'artigo': '41 a 47', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 2.180, de 2024',
  'artigos': [{'artigo': '48 a 56', 'incisos': [], 'parágrafos': []}],
  'anexos': []}]

In [73]:
from references_normalization import *

In [74]:
doc_matcher = legalDocumentsMatcher(DATA_PATH + "llama_deduplicated_embedded_and_annotated_reference_titles_20250217_remove_last_dup.pkl")

### Check the matches on the Q&A references, just to confirm

In [111]:
def check_question_reference_matches(question, which_field, question_references_dict, multiple_matches_list):
    if question[which_field] is not None:
        for i, reference in enumerate(question[which_field]):
            if 'título' in reference:
                reference_matches = doc_matcher.get_best_match(reference['título'])
        
                if len(reference_matches['best_matches']) == 1:
                    if reference_matches['best_matches'][0] not in question_references_dict['references']:
                        question_references_dict['references'][reference_matches['best_matches'][0]] = []
        
                    question_references_dict['references'][reference_matches['best_matches'][0]].append(reference)
                else:
                    print(f">>{question_references_dict['question_number']}, {which_field} item {i} multiple matches: {reference_matches}\n")
                    multiple_matches_list.append({
                        'question_number': question_references_dict['question_number'],
                        'reference_field': which_field,
                        'reference_item': i,
                        'reference_title': reference['título'],
                        'reference_matches': reference_matches['best_matches']
                    })
            else:
                print(f">> Missing reference title in entry {i}: {reference}")

In [82]:
matches = []

multiple_matches = []

for i, question in enumerate(questions):
    print(f"Handling question entry {i}")
    question_references = {}
    question_references['question_number'] = int(question['question_number'])
    question_references['references'] = {}

    check_question_reference_matches(question, 'formatted_references', question_references, multiple_matches)    
    check_question_reference_matches(question, 'formatted_embedded_references', question_references, multiple_matches)

    matches.append(question_references)

Handling question entry 0
Handling question entry 1
Handling question entry 2
Handling question entry 3
Handling question entry 4
Handling question entry 5
Handling question entry 6
Handling question entry 7
Handling question entry 8
Handling question entry 9
Handling question entry 10
Handling question entry 11
Handling question entry 12
Handling question entry 13
Handling question entry 14
Handling question entry 15
Handling question entry 16
Handling question entry 17
Handling question entry 18
Handling question entry 19
Handling question entry 20
Handling question entry 21
Handling question entry 22
Handling question entry 23
Handling question entry 24
Handling question entry 25
Handling question entry 26
Handling question entry 27
Handling question entry 28
>>29, formatted_references item 4 multiple matches: {'title_f1': 0.4, 'number_f1': 0, 'date_f1': 0, 'best_matches': ['Súmula nº 136 do Superior Tribunal de Justiça (STJ)', 'Lei nº 6.015, de 31 de dezembro de 1973', 'Lei nº 4.50

In [83]:
multiple_matches

[{'question_number': 29,
  'reference_field': 'formatted_embedded_references',
  'reference_item': 4,
  'reference_title': 'e-Financeira',
  'reference_matches': ['Súmula nº 136 do Superior Tribunal de Justiça (STJ)',
   'Lei nº 6.015, de 31 de dezembro de 1973',
   'Lei nº 4.506, de 30 de novembro de 1964',
   'Lei nº 4.591, de 16 de dezembro de 1964',
   'Lei nº 4.862, de 29 de novembro de 1965',
   'Lei nº 4.886, de 9 de dezembro de 1965',
   'Lei nº 5.172, de 25 de outubro de 1966 - Código Tributário Nacional (CTN)',
   'Lei nº 5.809, de 10 de outubro de 1972',
   'Lei nº 6.404, de 15 de dezembro de 1976',
   'Lei nº 4.242, de 1963',
   'Lei nº 6.766, de 19 de dezembro de 1979',
   'Lei nº 7.070, de 20 de dezembro de 1982',
   'Lei nº 7.290, de 19 de dezembro de 1984',
   'Lei nº 7.713, de 22 de dezembro de 1988',
   'Súmula nº 125 do Superior Tribunal de Justiça (STJ)',
   'Lei nº 8.023, de 12 de abril de 1990',
   'Lei nº 4.504, de 30 de novembro de 1964',
   'Lei nº 2.579, de 23

### Manually fixing the match issues in the Q&A document

Remove the "e-Financeira" reference.

In [113]:
questions[28]['formatted_embedded_references'].pop(4)

{'título': 'e-Financeira', 'artigos': [], 'anexos': []}

In [114]:
questions[28]['formatted_embedded_references']

[{'título': 'Declaração do Imposto sobre a Renda Retido na Fonte (Dirf)',
  'artigos': [],
  'anexos': []},
 {'título': 'Declaração de Serviços Médicos e de Saúde (Dmed)',
  'artigos': [],
  'anexos': []},
 {'título': 'Declaração de informações sobre Atividades Imobiliárias (Dimob)',
  'artigos': [],
  'anexos': []},
 {'título': 'Sistema de Recolhimento Mensal Obrigatório (Carnê-Leão)',
  'artigos': [],
  'anexos': []},
 {'título': 'Declaração sobre Operações Imobiliárias (DOI)',
  'artigos': [],
  'anexos': []},
 {'título': 'Declaração de Benefícios Fiscais (DFB)',
  'artigos': [],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 1.888, de 3 de maio de 2019',
  'artigos': [],
  'anexos': []}]

The following items also don't correspond to a proper reference and shall be removed:

1. 'Declaração do Imposto sobre a Renda Retido na Fonte (Dirf)'
1. 'Declaração de Serviços Médicos e de Saúde (Dmed)'
1. 'Declaração de informações sobre Atividades Imobiliárias (Dimob)'
1. 'Sistema de Recolhimento Mensal Obrigatório (Carnê-Leão)'
1. 'Declaração sobre Operações Imobiliárias (DOI)'
1. 'Declaração de Benefícios Fiscais (DFB)'

In [116]:
questions[28]['formatted_embedded_references'] = questions[28]['formatted_embedded_references'][-1:]

In [117]:
questions[28]['formatted_embedded_references']

[{'título': 'Instrução Normativa RFB nº 1.888, de 3 de maio de 2019',
  'artigos': [],
  'anexos': []}]

Fix the missing 'título' field, wrongly named 'tjomais'.

In [103]:
questions[140]['formatted_embedded_references']

[{'título': 'Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha',
  'artigos': [],
  'anexos': []},
 {'título': 'Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005',
  'artigos': [],
  'anexos': []},
 {'tjomais': 'Instrução Normativa SRF nº 208, de 27 de setembro de 2002',
  'artigos': [{'artigo': '15', 'incisos': [], 'parágrafos': ['1º']},
   {'artigo': '16', 'incisos': [], 'parágrafos': ['1º', '2º', '6º']}],
  'anexos': []}]

In [106]:
questions[140]['formatted_embedded_references'][2]['título'] = questions[140]['formatted_embedded_references'][2]['tjomais']

In [108]:
questions[140]['formatted_embedded_references'][2].pop('tjomais')

'Instrução Normativa SRF nº 208, de 27 de setembro de 2002'

In [109]:
questions[140]['formatted_embedded_references'][2]

{'artigos': [{'artigo': '15', 'incisos': [], 'parágrafos': ['1º']},
  {'artigo': '16', 'incisos': [], 'parágrafos': ['1º', '2º', '6º']}],
 'anexos': [],
 'título': 'Instrução Normativa SRF nº 208, de 27 de setembro de 2002'}

Remove 'e-CAC' reference.

In [118]:
questions[459]['formatted_references']

[{'título': 'Lei nº 9.250, de 26 de dezembro de 1995',
  'artigos': [{'artigo': '25', 'incisos': ['II'], 'parágrafos': ['1º']}]},
 {'título': 'Instrução Normativa RFB nº 1.888, de 3 de maio de 2019',
  'artigos': [],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 2.178, de 5 de março de 2024',
  'artigos': [{'artigo': '11', 'incisos': ['II'], 'parágrafos': ['2º']}]},
 {'título': 'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019',
  'artigos': [],
  'anexos': []},
 {'título': 'Solução de Consulta Cosit nº 214, de 20 de dezembro de 2021',
  'artigos': [],
  'anexos': []},
 {'título': 'Solução de Consulta Cosit nº 217, de 21 de setembro de 2023',
  'artigos': [],
  'anexos': []},
 {'título': 'Solução de Consulta Cosit nº 218, de 21 de setembro de 2023',
  'artigos': [],
  'anexos': []},
 {'título': 'e-CAC', 'artigos': [], 'anexos': []}]

In [120]:
questions[459]['formatted_references'] = questions[459]['formatted_references'][:-1]

In [121]:
questions[459]['formatted_references']

[{'título': 'Lei nº 9.250, de 26 de dezembro de 1995',
  'artigos': [{'artigo': '25', 'incisos': ['II'], 'parágrafos': ['1º']}]},
 {'título': 'Instrução Normativa RFB nº 1.888, de 3 de maio de 2019',
  'artigos': [],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 2.178, de 5 de março de 2024',
  'artigos': [{'artigo': '11', 'incisos': ['II'], 'parágrafos': ['2º']}]},
 {'título': 'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019',
  'artigos': [],
  'anexos': []},
 {'título': 'Solução de Consulta Cosit nº 214, de 20 de dezembro de 2021',
  'artigos': [],
  'anexos': []},
 {'título': 'Solução de Consulta Cosit nº 217, de 21 de setembro de 2023',
  'artigos': [],
  'anexos': []},
 {'título': 'Solução de Consulta Cosit nº 218, de 21 de setembro de 2023',
  'artigos': [],
  'anexos': []}]

Fix the last embedded reference, which implicitly refers to the law number present in the question, but not in the answer.

In [122]:
questions[516]['formatted_embedded_references']

[{'título': 'Lei',
  'artigos': [{'artigo': '3º', 'incisos': ['III'], 'parágrafos': ['1º']}],
  'anexos': []}]

In [124]:
questions[516]['formatted_embedded_references'][0]['título']

'Lei'

In [126]:
questions[516]['formatted_embedded_references'][0]['título'] = 'Lei nº 11.524, de 24 de setembro de 2007'

In [127]:
questions[516]['formatted_embedded_references']

[{'título': 'Lei nº 11.524, de 24 de setembro de 2007',
  'artigos': [{'artigo': '3º', 'incisos': ['III'], 'parágrafos': ['1º']}],
  'anexos': []}]

#### Save the fixed questions version

In [128]:
with open(DATA_PATH + "extracted_questions_with_embedded_references_20250217_final_match_fixes.pkl", "wb") as output_file:
    pickle.dump(questions, output_file, pickle.HIGHEST_PROTOCOL)